In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.sparse.linalg import lsmr
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve

# Define constants
C = 1.0  # You can change this value
T = 1.0  # Upper time limit
t_max = 0.9  # Time at which we evaluate
N = 100  # Number of discrete steps
x = 0.5  # This is the x from the RHS (Gaussian)

# Discretize the time interval
tau_vals = np.linspace(0, t_max, N)
delta_tau = tau_vals[1] - tau_vals[0]

# Define h(t) = C / sqrt(T - t)
def h(t):
    return C / np.sqrt(T - t)

# Define the kernel K(t, tau)
def kernel(t, tau):
    term1 = 2 / np.sqrt(2 * np.pi * (t - tau))
    h_diff = h(t) - h(tau)
    term2 = np.exp(-(h_diff ** 2) / (2 * (t - tau)))
    return term1 * term2

# Build the kernel matrix A
A = np.zeros((N, N))
for i in range(N):
    for j in range(N):
        if tau_vals[j] < tau_vals[i]:  # Only valid when tau_j < tau_i
            A[i, j] = kernel(tau_vals[i], tau_vals[j]) * delta_tau

# Right-hand side: Gaussian
rhs = (1 / np.sqrt(2 * np.pi * t_max)) * np.exp(-x ** 2 / (2 * t_max))

# Solve the linear system A * d_h = rhs
# We replicate the RHS to match the dimensions of A for numerical solving
rhs_vec = np.full(N, rhs)

# Solve the system to find d_h
d_h = solve(A, rhs_vec)

# Plot the results
plt.plot(tau_vals, d_h, label="Solution for $d_h(\\tau)$")
plt.xlabel("tau")
plt.ylabel("$d_h(\\tau)$")
plt.legend()
plt.title("Numerical Solution for $d_h(\\tau)$")
plt.show()


''# Define h(t) = C as a constant value
def constant_h(t, C):
    return C

# Modify the kernel to use constant h(t) = C
def constant_kernel(t, tau, C, epsilon=1e-10):
    diff = t - tau
    if diff < epsilon:
        diff = epsilon  # To avoid division by zero or very small values
    term1 = 2 / np.sqrt(2 * np.pi * diff)
    h_diff = constant_h(t, C) - constant_h(tau, C)
    term2 = np.exp(-(h_diff ** 2) / (2 * diff))
    return term1 * term2

# Let's define a range of probabilities to compute corresponding values of C
probabilities = [0.55, 0.60, 0.65, 0.70, 0.75]
colors = ['b', 'g', 'r', 'c', 'm']  # Color for each plot

# Plot the results for each probability
plt.figure(figsize=(10, 6))

for i, p in enumerate(probabilities):
    # Compute C for each probability
    C = norm.ppf(p)

    # Rebuild the kernel matrix using the updated constant kernel
    A_stable = np.zeros((N, N))
    for row in range(N):
        for col in range(N):
            if tau_vals[col] < tau_vals[row]:  # Only valid when tau_j < tau_i
                A_stable[row, col] = constant_kernel(tau_vals[row], tau_vals[col], C) * delta_tau

    # Apply regularization again to the modified matrix
    A_stable_reg = A_stable + regularization_param * np.eye(N)

    # Solve the system using the least-squares solver with the new constant C
    d_h_lsmr = lsmr(A_stable_reg, rhs_vec, atol=1e-8, btol=1e-8, maxiter=10000)[0]  # Increased tolerance and iterations

    # Plot the results after applying the least-squares solver with the constant C
    plt.plot(tau_vals, d_h_lsmr, color=colors[i], label=f"p = {p}, C = {C:.4f}")

# Set labels and title
plt.xlabel("tau")
plt.ylabel("$d_h(\\tau)$")
plt.title("Solution for $d_h(\\tau)$ with Constant $h(t) = C$ for Various Probabilities")
plt.legend()
plt.grid(True)
plt.show()


LinAlgError: Matrix is singular.